# Tabell til kontrollside
Kontrollér-siden i Edith krever at man har forhåndsprogrammert kontroller i et visst format. I denne notebooken lager jeg en eksempelsnutt for hvordan en slik tabell kan lages. Målet er at det skal være enkelt for statistikkfaglig å følge eksempelet og skrive ut egne kontrolltabeller. Jeg velger å gjøre dette ved bruk av R, men det kan like greit gjøres med andre verktøy.

For at man skal kunne kjøre denne notebooken, må database med rådatatabell være opprettet på forhånd. Se "oppretting_database_raadata.ipynb". I tillegg må config.json og variabler.json være opprettet.

## Importerer pakker, kobler opp til SQL-base og innhenter config-fil

In [ ]:

suppressPackageStartupMessages(library(tidyverse)) #OBS: suppress-funksjonen stopper advarsler forbundet med innlasting av denne pakken.
library(RSQLite)
library(DBI)
library(rjson)


config <- fromJSON(file = "config.json")
variabler <- fromJSON(file = "variabler.json")

#Bestemmer perioder basert på config-fil
t <- config$perioder$t$periode
t_1 <- config$perioder$`t-1`$periode
t_2 <- config$perioder$`t-2`$periode

print(t)
print(t_1)
print(t_2)

con <- dbConnect(RSQLite::SQLite(), ":memory:", dbname=paste0(config$data$filsti,"/edith.sqlite"))

# Innhent rådatatabell

In [ ]:
raadata <- dbReadTable(con, "raadata") %>% select(-index)
dbDisconnect(con)
head(raadata)

## Velger ut relevante numeriske variabler fra JSON-fil
Edith fungerer for øyeblikket best for numeriske variabler. Koden vil virke dårlig med kategoriske variabler og vil ikke fungere dersom det ligger kategoriske variabler i variabler.json-filen.

In [ ]:
raadata2 <- raadata[raadata$VARIABEL %in% variabler$variabler,]

#Erstatter komma med punktum i verdikolonnene og gjør disse numeriske - nødvendig pga. måten data leses inn fra Dynarev.
raadata3 <- raadata2 %>%
    #OBS: her selekteres variabler på bakgrunn av om string starter med År_. Må endres dersom annerledes i config
    mutate_at(vars(starts_with("År_")), ~str_replace(., ",", ".")) %>% 
    mutate_at(vars(starts_with("År_")), as.numeric)

In [ ]:
beregnet_diff <- raadata3 %>%
    mutate(DIFF = eval(as.name(t)) - eval(as.name(t_1)),
           DIFF_PST = DIFF/eval(as.name(t_1)))

In [ ]:
raadata_pivotert <- beregnet_diff %>%
                #select(ORGNR, VARIABEL, matches("År"), DIFF, DIFF_PST) %>%
                distinct(ORGNR, VARIABEL, .keep_all = TRUE) %>% #Fant en dublett på intfou på et orgnr
                pivot_wider(id_cols = c("ORGNR", "NAVN", "NACE1"),
                            names_from = "VARIABEL",
                            values_from = c(all_of(t), all_of(t_1), all_of(t_2), DIFF, DIFF_PST)) %>%
                rename_at(vars(starts_with(t)), ~str_remove(., paste0(t,"_")))

## Lager feillister
#### Alternativ 1: hardkoder hver enkelt variabelnavn

In [ ]:
intfou_stor_diff <- raadata_pivotert %>%
            filter(DIFF_PST_INTFOU > 1) %>%
            mutate(feilliste = "FoU-kostnader store differanser") %>%
            select(feilliste, ORGNR, NAVN, matches("INTFOU"), NACE1)

fouars_stor_diff <- raadata_pivotert %>%
    filter(DIFF_PST_FOUARS > 1) %>%
    mutate(feilliste = "FoU-årsverk store differanser") %>%
    select(feilliste, ORGNR, NAVN, matches("FOUARS"), NACE1)

feilliste_master <- intfou_stor_diff %>%
          bind_rows(fouars_stor_diff) %>%
          mutate(kommentar = '') %>%
          select(kommentar, feilliste, everything())

print(feilliste_master)

#### Alternativ 2
Lager en funksjon som lister ut alle enheter+variabler med endringer på over 100 % sammenlignet med forrige periode

In [ ]:
kontroller_stor_endring <- function(variabel){
    raadata_pivotert %>%
        select(ORGNR, NAVN, matches(variabel), NACE1) %>%
        filter_at(vars(paste0("DIFF_PST_",variabel)), all_vars(. > 1)) %>%
        mutate(feilliste = paste0(variabel, "_stor_diff"))
}

In [ ]:
#lager en feilliste ved å kjøre funksjonen for én variabel (fnutter rundt). INTFOU er en variabel fra FoU-undersøkelsen.
kontroller_stor_endring("INTFOU") %>% head()

In [ ]:
#Lister opp numeriske variabler
num_variabler <- unique(raadata3$VARIABEL)

#For-løkke for å kjøre funksjonen for alle numeriske variabler og slå disse sammen til en masterfil
feilliste_master <- data.frame()
for (i in num_variabler){
  kontroll_endring <- kontroller_stor_endring(i)
  feilliste_master <- bind_rows(feilliste_master, kontroll_endring)
}

dim(feilliste_master)

In [ ]:
feilliste_master <- feilliste_master %>%
  mutate(kommentar = "") %>%
  select(kommentar, feilliste, everything())

In [ ]:
#Appen takler et begrenset antall kolonner. Tester derfor med de første 50.
feilliste <- feilliste_master %>% select(1:50)

## Lagrer feilliste på område hvor feillisten skal innhentes fra i Edith

In [ ]:
#Filsti angis av config-fil
write.csv(feilliste_master, paste0(config$data$filsti, "/feilliste.csv"), row.names = FALSE)